In [1]:
function loggingProxy(o, objName) {
  const handler = {
    get(o, name, receiver) {
      console.log(`Handler get(${objName}, ${name.toString()})`)
      let value = Reflect.get(o, name, receiver)
      // 如果属性是目标的自由属性, 而且值为对象或函数, 则返回这个值的代理
      if(Reflect.ownKeys(o).includes(name) && 
      (typeof value == 'object' || typeof value == 'function')) {
        return loggingProxy(value, `${objName}.${name.toString()}`)
      }
      // 否则原封不动返回值
      return value
    },
    set(o, name, value, receiver) {
      console.log(`Handler set(${objName}, ${name.toString()}, ${value})`)
      return Reflect.set(o, name, value, receiver)
    },
    apply(target, thisArgs, args) {
      console.log(`Handler ${objName}(${JSON.stringify(args)})`)
      return Reflect.apply(target, thisArgs, args)
    },
    construct(target, argsList, newTarget) {
      console.log(`Handler construct(${target}, ${argsList})`)
      return Reflect.construct(target, argsList, newTarget)
    }
  }

  /* 可以通过反射API直接生成, 应为属性名字是对应的 */
  Reflect.ownKeys(Reflect).forEach(handlerName => {
    if(!(handlerName in handler)) {
      handler[handlerName] = function(target, ...args) {
        console.log(`Handler ${handlerName}(${objName}, ${args})`)
        return Reflect[handlerName](target, ...args)
      }
    }
  })

  return new Proxy(o, handler)
}

// 这个方法可以让我看到所有API方法处理数据的过程
let data = [10, 20]
let methods = {square(x) { return x*x} }

let proxyData = loggingProxy(data, 'data')
let proxyMethods = loggingProxy(methods, 'methods')

/* 这里可以看到 map 方法执行过程 */
proxyData.map(methods.square)

/* 这里可以看到 square 方法执行的过程 */
data.map(proxyMethods.square) 

/* 这里可以看到迭代过程 */
for(let x of proxyData) console.log('data', x)

Handler get(data, map)
Handler get(data, length)
Handler get(data, constructor)
Handler has(data, 0)
Handler get(data, 0)
Handler has(data, 1)
Handler get(data, 1)
Handler get(methods, square)
Handler methods.square([10,0,[10,20]])
Handler methods.square([20,1,[10,20]])
Handler get(data, Symbol(Symbol.iterator))
Handler get(data, length)
Handler get(data, 0)
data 10
Handler get(data, length)
Handler get(data, 1)
data 20
Handler get(data, length)
